# Deepki's Technical assessment

## Imports and acquisition of data


In [2]:
import pandas as pd 
import geopandas as gpd
import requests
import gzip

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


Connecting to the website


In [2]:


import requests

url = "https://sites.research.google/open-buildings"
response = requests.get(url)
print(f"Response Status Code: {response.status_code}")

Response Status Code: 200


In [ ]:

url = "https://storage.googleapis.com/open-buildings-data/v3/polygons_s2_level_4_gzip/009_buildings.csv.gz"
response = requests.get(url)

# Proceed only if the connection is successful
response.raise_for_status()

# Step 2: Save the gzipped file locally
with open('building_data.gz', 'wb') as f:
    f.write(response.content)

# Step 3: Decompress the gzipped file
with gzip.open('building_data.gz', 'rb') as f:
    file_content = f.read()

# Step 4: Save the decompressed file locally 
with open('building_data.csv', 'wb') as f:
    f.write(file_content)




## Finding the closest building

Visualising the first lines of data

In [3]:

df=pd.read_csv("building_data.csv")
print(df.head(10))

    latitude  longitude  area_in_meters  confidence  \
0 -22.901624 -42.929226         56.3454      0.8570   
1 -23.205527 -44.720797         30.7992      0.7711   
2 -22.726750 -42.825261         32.4435      0.7943   
3 -22.723037 -43.357391         57.6590      0.8026   
4 -22.479556 -44.359011        181.2616      0.7694   
5 -22.785605 -43.434637         48.5324      0.7073   
6 -22.873635 -43.023156         38.4617      0.7079   
7 -22.643436 -43.252496         12.1259      0.6590   
8 -22.761967 -43.342515         15.1879      0.7134   
9 -22.699793 -43.136531         89.5987      0.7478   

                                            geometry full_plus_code  
0  POLYGON((-42.9291743293887 -22.9016302192089, ...  589V33XC+9825  
1  POLYGON((-44.7207593467604 -23.205536574256, -...  588QQ7VH+QMPV  
2  POLYGON((-42.8252088612076 -22.7267550999572, ...  589V75FF+7VXW  
3  POLYGON((-43.3573777886799 -22.7230910476831, ...  589R7JGV+Q2QF  
4  POLYGON((-44.3589553775915 -22.4796307443

Find the closest coordinates

In [13]:
Cristo_Coordinates=[-22.950996196, -43.206499174]

def get_closest_building(target_coordinates=Cristo_Coordinates, building_df=df):
    # Calculate absolute differences in latitude and longitude
    building_df['lat_diff'] = abs(building_df['latitude'] - target_coordinates[0])
    building_df['lon_diff'] = abs(building_df['longitude'] - target_coordinates[1])

    # Find the index of the building with the minimum absolute differences
    closest_building_index = (building_df['lat_diff'] + building_df['lon_diff']).idxmin()

    # Retrieve the full plus code and differences of the closest building, the differences and its coordinates
    closest_building_plus_code = building_df.loc[closest_building_index, 'full_plus_code']

    closest_building_lat_diff = building_df.loc[closest_building_index, 'lat_diff']
    closest_building_lon_diff = building_df.loc[closest_building_index, 'lon_diff']
    
    closest_building_lat = building_df.loc[closest_building_index, 'latitude']
    closest_building_lon = building_df.loc[closest_building_index, 'longitude']

    print(f"Latitude difference: {closest_building_lat_diff} degrees")
    print(f"Longitude difference: {closest_building_lon_diff} degrees")
    print(f"Latitude of closest building: {closest_building_lat} degrees")
    print(f"Longitude of closest building: {closest_building_lon} degrees")
    return [closest_building_plus_code,closest_building_lat, closest_building_lon]

closest_building_plus_code=get_closest_building(Cristo_Coordinates, df)[0]


Latitude difference: 0.0008792940000006411 degrees
Longitude difference: 0.004484976000000529 degrees
Latitude of closest building: -22.95187549 degrees
Longitude of closest building: -43.21098415 degrees


## Visualization


In [17]:
import folium
lat=get_closest_building(Cristo_Coordinates, df)[1]
lon=get_closest_building(Cristo_Coordinates, df)[2]




m = folium.Map(location=[Cristo_Coordinates[0], Cristo_Coordinates[1]], zoom_start=15)
folium.Marker([Cristo_Coordinates[0], Cristo_Coordinates[1]],
              popup="Cristo Redentor\nCoordinates").add_to(m)
folium.Marker([lat, lon],
                  popup=f"Closest Building\nFull Plus Code: {closest_building_plus_code}").add_to(m)

m

Latitude difference: 0.0008792940000006411 degrees
Longitude difference: 0.004484976000000529 degrees
Latitude of closest building: -22.95187549 degrees
Longitude of closest building: -43.21098415 degrees
Latitude difference: 0.0008792940000006411 degrees
Longitude difference: 0.004484976000000529 degrees
Latitude of closest building: -22.95187549 degrees
Longitude of closest building: -43.21098415 degrees
